In [112]:
# Bring in all of our dependencies
from keras.models import Sequential
from keras.layers import Dense
import numpy as np
import gym
import math



In [227]:
# We should define the environment here and detremine the sizing so we can generalize our simple neural network
env    = gym.make('CartPole-v0')   # Choose our environment
tmp    = env.action_space.sample()
print(type(tmp))
action = np.array([tmp])
N      = len(action)               # Use this to generalize the input size for the NN
print(N)

[2016-11-24 08:28:52,688] Making new env: CartPole-v0


<class 'int'>
1


In [232]:
# Make a very simple neural network, reLU layers have been shown to train faster. We only have one hidden layer here, very simple
model = Sequential()
model.add(Dense(100, input_shape=((4,)), activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(2))

In [233]:
# Build the model for training, we can tweak our loss function and optimizer for different environments
model.compile(loss='mean_squared_error', optimizer='sgd')

In [249]:
# Initialize replay memory
#repMem = np.empty((0,4))
gamma = 0.5
epsUpdate = 0.1
obs1 = env.reset()
action = env.action_space.sample()
obs2, reward, done, info = env.step(action)
repMem = np.array([obs1,action,reward,obs2])

#repMem = [1,2,3,4]
# Run episodes tally results
for i_episode in range(100):
    obs1 = env.reset()
    epsilon = epsUpdate * i_episode
    rTally = np.array([0])
    loss = 0.
    
    for t in range(300):
        env.render()
        obs2 = obs1
        # Add action selection by max output of network over time, update epsilon slowly
        if np.random.uniform(0,1,1) >= (epsilon):
            action = env.action_space.sample()                    # this is a random action
        else:
            ta = model.predict(obs2[np.newaxis,])
            action = int(np.argmax(model.predict(obs2[np.newaxis,])[0]))
            print(ta[0])
        
        
        obs1, reward, done, info = env.step(action)    # apply the action to the model and check the outputs
        
        repMem = np.column_stack((repMem, np.array([obs2,action,reward,obs1])))
            
        if repMem.shape[1] > 10:
            P = max(float(1),math.floor(repMem.shape[1]/2))
        else:
            P = int(1)

        # Get random batch
        memBatch = repMem[:,np.random.choice(repMem.shape[1], int(P), replace=False)]
        M = int(P)
        
        # Put Q-Learning stuff here to start...
        # Q[s,a] = Q[s,a] + a(r + y max_a'(Q[s',a'] - Q[s,a]))
        # Get targets
        tgt = np.zeros((M,2))

        for batchNum in range(M):
            #print(batchNum)
            
            tgt[batchNum,:] = model.predict(tmp)[0]
            tmp = memBatch[3,batchNum]
            tmp = tmp[np.newaxis,]
            QVal = np.max(model.predict(tmp)[0])
            tmpReward = memBatch[2,batchNum]
            tmpA      = memBatch[1,batchNum]
            if done:
                tgt[batchNum,tmpA] = tmpReward
            else:
                tgt[batchNum,tmpA] = tmpReward + (gamma * QVal)
                
                              
        # End batch for
        #print(type(memBatch[np.newaxis,0]))
        # Train              
        if memBatch[0,:].shape[0] == 1:
            tmp = memBatch[0,]
            tmp = tmp[0]
            tmp = tmp[np.newaxis,]
        else:
            tmp = np.vstack(memBatch[0,:])
        
        rTally = rTally + reward
        loss += model.train_on_batch(tmp, tgt)
        #print(loss)
        if done:
            print("Episode finished after {} timesteps".format(t+1))
            break

Episode finished after 26 timesteps
[ 2.03160858  1.87815118]
Episode finished after 18 timesteps
[ 2.05522919  1.90620279]
[ 1.89694643  1.93267453]
[ 2.0270021   1.88323176]
[ 1.9803493  1.8828243]
[ 1.89650607  1.8874259 ]
[ 1.88664091  1.87178278]
[ 1.85703242  1.81858444]
[ 1.88587928  1.80047345]
Episode finished after 22 timesteps
[ 1.85952389  1.96828973]
[ 1.85558152  1.95687258]
[ 1.96628594  1.90234184]
[ 1.91867232  1.92452168]
[ 1.97444093  1.89970922]
[ 1.98699152  1.89161968]
Episode finished after 14 timesteps
[ 1.8583442   1.88529062]
[ 1.84792995  1.92431569]
[ 1.9023788   1.88554621]
[ 1.8606081   1.90366507]
[ 1.8637929   1.86424553]
Episode finished after 16 timesteps
[ 1.85807061  1.87253761]
[ 1.85284781  1.93656754]
[ 1.8867873   1.83169448]
[ 1.93140888  1.84997761]
[ 2.00999808  1.9024502 ]
[ 2.08887315  1.95544529]
[ 2.18630338  2.0226481 ]
[ 2.21749282  2.04701328]
Episode finished after 14 timesteps
[ 1.82759333  1.95217931]
[ 1.86285615  1.97638798]
[ 1.84

KeyboardInterrupt: 